# <font color =blue> <center>__Capstone Project Final Assignment Notebook__</font></center>

## Notebook information

The following notebook contains the development and analysisis of the desired location to open an experience center of Spanish wine and food.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Import the NY Neighborhood data

In [2]:
#Downlad the data

!wget -q -O 'nyu_2451_34572-geojson.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

#Explore the infodmration in .json 

with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)
    
newyork_data

/bin/sh: wget: command not found
Data downloaded!


{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [3]:
# Transform .json into a Pandas DF

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()    

Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

### Create a Map of New York with the neighborhoods downloaded previously

This is usefull in order to show in the report the different neighborhoods in NY from which I'll select the desired location.

In [4]:
# Using geopy to get the latitude and longitude values of NY

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values using the Folium library
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Using Foursquare data to explore neighborhoods in the dataframe

First the Foursquare credentials must be defined in order to import the data based on the coordinates defined above (coordinates of each neighborhood).

This will return the list of all the venues for each neighborhood which will be filtered to show only the needed ones:

- Wine Shops
- Wine Bars
- Paella Restaurants
- Spanish Restaurants
- Tapas Restaurant


In [5]:
#Define Foursquare Credentianls and Version
CLIENT_ID = 'SDKWCH31F0XWILJ4Y4ZRPZVE5APPBKXJIU05LSPFX45OPHUR' # your Foursquare ID
CLIENT_SECRET = 'BA2XABVGHNGK44QTAJ4XB1N1E2J5WZRTFGNRIRTCSJKEQITC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


#Try first with the first neighborhood

neighborhoods.loc[0, 'Neighborhood']

neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

Your credentails:
CLIENT_ID: SDKWCH31F0XWILJ4Y4ZRPZVE5APPBKXJIU05LSPFX45OPHUR
CLIENT_SECRET:BA2XABVGHNGK44QTAJ4XB1N1E2J5WZRTFGNRIRTCSJKEQITC
Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.
9 venues were returned by Foursquare.


/Users/sebasjunca/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [6]:
# Create a function to explore all neighborhoods in NY City

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [7]:
print(ny_venues.shape)
ny_venues.head()

(10145, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Wakefield              40.894705              -73.847201   
1    Wakefield              40.894705              -73.847201   
2    Wakefield              40.894705              -73.847201   
3    Wakefield              40.894705              -73.847201   
4    Wakefield              40.894705              -73.847201   

              Venue  Venue Latitude  Venue Longitude  Venue Category  
0  Lollipops Gelato       40.894123       -73.845892    Dessert Shop  
1          Rite Aid       40.896649       -73.844846        Pharmacy  
2  Carvel Ice Cream       40.890487       -73.848568  Ice Cream Shop  
3         Walgreens       40.896528       -73.844700        Pharmacy  
4           Dunkin'       40.890459       -73.849089      Donut Shop

In [8]:
### Venues per neighborhoods in NY City

ny_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  \
Neighborhood                                                               
Allerton                                      32                      32   
Annadale                                      13                      13   
Arden Heights                                  5                       5   
Arlington                                      4                       4   
Arrochar                                      19                      19   
Arverne                                       18                      18   
Astoria                                      100                     100   
Astoria Heights                               12                      12   
Auburndale                                    19                      19   
Bath Beach                                    50                      50   
Battery Park City                             77                      77   
Bay Ridge                                     84                      84   
Bay Terrace                                   48                      48   
Baychester                                    22                      22   
Bayside                                       71                      71   
Bayswater                                      2                       2   
Bedford Park                                  33                      33   
Bedford Stuyvesant                            25                      25   
Beechhurst                                    12                      12   
Bellaire                                      10                      10   
Belle Harbor                                  17                      17   
Bellerose                                     21                      21   
Belmont                                       96                      96   
Bensonhurst                                   34                      34   
Bergen Beach                                   6                       6   
Blissville                                    20                      20   
Bloomfield                                     5                       5   
Boerum Hill                                   90                      90   
Borough Park                                  21                      21   
Breezy Point                                   4                       4   
Briarwood                                      8                       8   
Brighton Beach                                45                      45   
Broad Channel                                  6                       6   
Broadway Junction                             16                      16   
Bronxdale                                     12                      12   
Brooklyn Heights                             100                     100   
Brookville                                     2                       2   
Brownsville                                   20                      20   
Bulls Head                                    43                      43   
Bushwick                                      72                      72   
Butler Manor                                   6                       6   
Cambria Heights                               13                      13   
Canarsie                                       6                       6   
Carnegie Hill                                 94                      94   
Carroll Gardens                              100                     100   
Castle Hill                                   10                      10   
Castleton Corners                             16                      16   
Central Harlem                                41                      41   
Charleston                                    30                      30   
Chelsea                                      105                     105   
Chinatown                                    100                     100   

In [9]:
# Analyze each neighborhood

# one hot encoding
newyork_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name = "Neighborhood"
first_col = newyork_onehot.pop(col_name)
newyork_onehot.insert(0, col_name, first_col)

newyork_onehot.head()

Neighborhood  Accessories Store  Acupuncturist  Adult Boutique  \
0    Wakefield                  0              0               0   
1    Wakefield                  0              0               0   
2    Wakefield                  0              0               0   
3    Wakefield                  0              0               0   
4    Wakefield                  0              0               0   

   Afghan Restaurant  African Restaurant  American Restaurant  Antique Shop  \
0                  0                   0                    0             0   
1                  0                   0                    0             0   
2                  0                   0                    0             0   
3                  0                   0                    0             0   
4                  0                   0                    0             0   

   Arcade  Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0       0                 0                       0            0           0   
1       0                 0                       0            0           0   
2       0                 0                       0            0           0   
3       0                 0                       0            0           0   
4       0                 0                       0            0           0   

   Arts & Crafts Store  Arts & Entertainment  Asian Restaurant  \
0                    0                     0                 0   
1                    0                     0                 0   
2                    0                     0                 0   
3                    0                     0                 0   
4                    0                     0                 0   

   Athletics & Sports  Auditorium  Australian Restaurant  Austrian Restaurant  \
0                   0           0                      0                    0   
1                   0           0                      0                    0   
2                   0           0                      0                    0   
3                   0           0                      0                    0   
4                   0           0                      0                    0   

   Automotive Shop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0                0          0           0           0       0     0    0   
1                0          0           0           0       0     0    0   
2                0          0           0           0       0     0    0   
3                0          0           0           0       0     0    0   
4                0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Court  Bath House  Beach  \
0               0                 0                 0           0      0   
1               0                 0                 0           0      0   
2               0                 0                 0           0      0   
3               0                 0                 0           0      0   
4               0                 0                 0           0      0   

   Beach Bar  Bed & Breakfast  Beer Bar  Beer Garden  Beer Store  \
0          0                0         0            0           0   
1          0                0         0            0           0   
2          0                0         0            0           0   
3          0                0         0            0           0   
4          0                0         0            0           0   

   Big Box Store  Bike Rental / Bike Share  Bike Shop  Bike Trail  Bistro  \
0              0                         0          0           0       0   
1              0                         0          0           0       0   
2              0                         0          0           0       0   
3              0                         0          0           0       0   
4              0                         0   

In [10]:
newyork_onehot.shape

(10145, 426)

In [11]:
ny_spanish_venue = newyork_onehot.filter(['Neighborhood', 'Wine Bar', 'Wine Shop', 'Tapas Restaurant', 'Spanish Restaurant', 'Paella Restaurant'])

ny_spanish_venue.head()

Neighborhood  Wine Bar  Wine Shop  Tapas Restaurant  Spanish Restaurant  \
0    Wakefield         0          0                 0                   0   
1    Wakefield         0          0                 0                   0   
2    Wakefield         0          0                 0                   0   
3    Wakefield         0          0                 0                   0   
4    Wakefield         0          0                 0                   0   

   Paella Restaurant  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0

In [12]:
ny_spa_grouped = ny_spanish_venue.groupby('Neighborhood').mean().reset_index()

ny_spa_grouped

Neighborhood  Wine Bar  Wine Shop  Tapas Restaurant  \
0                     Allerton  0.000000   0.000000          0.000000   
1                     Annadale  0.000000   0.000000          0.000000   
2                Arden Heights  0.000000   0.000000          0.000000   
3                    Arlington  0.000000   0.000000          0.000000   
4                     Arrochar  0.000000   0.000000          0.000000   
5                      Arverne  0.000000   0.055556          0.000000   
6                      Astoria  0.000000   0.010000          0.000000   
7              Astoria Heights  0.000000   0.000000          0.000000   
8                   Auburndale  0.000000   0.000000          0.000000   
9                   Bath Beach  0.000000   0.000000          0.000000   
10           Battery Park City  0.000000   0.025974          0.000000   
11                   Bay Ridge  0.000000   0.000000          0.000000   
12                 Bay Terrace  0.000000   0.000000          0.000000   
13                  Baychester  0.000000   0.000000          0.000000   
14                     Bayside  0.014085   0.000000          0.000000   
15                   Bayswater  0.000000   0.000000          0.000000   
16                Bedford Park  0.000000   0.000000          0.000000   
17          Bedford Stuyvesant  0.040000   0.040000          0.000000   
18                  Beechhurst  0.000000   0.000000          0.000000   
19                    Bellaire  0.000000   0.000000          0.000000   
20                Belle Harbor  0.000000   0.000000          0.000000   
21                   Bellerose  0.000000   0.047619          0.000000   
22                     Belmont  0.000000   0.000000          0.000000   
23                 Bensonhurst  0.000000   0.000000          0.000000   
24                Bergen Beach  0.000000   0.000000          0.000000   
25                  Blissville  0.000000   0.000000          0.000000   
26                  Bloomfield  0.000000   0.000000          0.000000   
27                 Boerum Hill  0.000000   0.011111          0.000000   
28                Borough Park  0.000000   0.000000          0.000000   
29                Breezy Point  0.000000   0.000000          0.000000   
30                   Briarwood  0.000000   0.000000          0.000000   
31              Brighton Beach  0.000000   0.000000          0.000000   
32               Broad Channel  0.000000   0.000000          0.000000   
33           Broadway Junction  0.000000   0.000000          0.000000   
34                   Bronxdale  0.000000   0.000000          0.000000   
35            Brooklyn Heights  0.010000   0.020000          0.000000   
36                  Brookville  0.000000   0.000000          0.000000   
37                 Brownsville  0.000000   0.000000          0.000000   
38                  Bulls Head  0.000000   0.000000          0.000000   
39                    Bushwick  0.000000   0.000000          0.000000   
40                Butler Manor  0.000000   0.000000          0.000000   
41             Cambria Heights  0.000000   0.000000          0.000000   
42                    Canarsie  0.000000   0.000000          0.000000   
43               Carnegie Hill  0.010638   0.031915          0.000000   
44             Carroll Gardens  0.020000   0.030000          0.000000   
45                 Castle Hill  0.000000   0.000000          0.000000   
46           Castleton Corners  0.000000   0.000000          0.000000   
47              Central Harlem  0.000000   0.000000          0.024390   
48                  Charleston  0.000000   0.000000          0.000000   
49                     Chelsea  0.009524   0.009524          0.009524   
50                   Chinatown  0.000000   0.000000          0.000000   
51                 City Island  0.000000   0.000000          0.000000   
52                   City Line  0.000000   0.000000          0.000000   
53                Civic Center  0.010000   0.020000          0.000000   
54 

In [13]:
#Print the neighborhoods with the top 5 most common venues

num_top_venues = 5

for hood in ny_spa_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_spa_grouped[ny_spa_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Allerton----
                venue  freq
0  Spanish Restaurant  0.03
1            Wine Bar  0.00
2           Wine Shop  0.00
3    Tapas Restaurant  0.00
4   Paella Restaurant  0.00


----Annadale----
                venue  freq
0            Wine Bar   0.0
1           Wine Shop   0.0
2    Tapas Restaurant   0.0
3  Spanish Restaurant   0.0
4   Paella Restaurant   0.0


----Arden Heights----
                venue  freq
0            Wine Bar   0.0
1           Wine Shop   0.0
2    Tapas Restaurant   0.0
3  Spanish Restaurant   0.0
4   Paella Restaurant   0.0


----Arlington----
                venue  freq
0            Wine Bar   0.0
1           Wine Shop   0.0
2    Tapas Restaurant   0.0
3  Spanish Restaurant   0.0
4   Paella Restaurant   0.0


----Arrochar----
                venue  freq
0            Wine Bar   0.0
1           Wine Shop   0.0
2    Tapas Restaurant   0.0
3  Spanish Restaurant   0.0
4   Paella Restaurant   0.0


----Arverne----
                venue  freq
0           Win

In [14]:
#Orginize the data in decending order and create a Pandas DF

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_spa_grouped['Neighborhood']

for ind in np.arange(ny_spa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_spa_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0                     Allerton    Spanish Restaurant     Paella Restaurant   
1                     Annadale     Paella Restaurant    Spanish Restaurant   
2                Arden Heights     Paella Restaurant    Spanish Restaurant   
3                    Arlington     Paella Restaurant    Spanish Restaurant   
4                     Arrochar     Paella Restaurant    Spanish Restaurant   
5                      Arverne             Wine Shop     Paella Restaurant   
6                      Astoria             Wine Shop     Paella Restaurant   
7              Astoria Heights     Paella Restaurant    Spanish Restaurant   
8                   Auburndale     Paella Restaurant    Spanish Restaurant   
9                   Bath Beach    Spanish Restaurant     Paella Restaurant   
10           Battery Park City             Wine Shop     Paella Restaurant   
11                   Bay Ridge     Paella Restaurant    Spanish Restaurant   
12                 Bay Terrace     Paella Restaurant    Spanish Restaurant   
13                  Baychester    Spanish Restaurant     Paella Restaurant   
14                     Bayside    Spanish Restaurant              Wine Bar   
15                   Bayswater     Paella Restaurant    Spanish Restaurant   
16                Bedford Park    Spanish Restaurant     Paella Restaurant   
17          Bedford Stuyvesant             Wine Shop              Wine Bar   
18                  Beechhurst     Paella Restaurant    Spanish Restaurant   
19                    Bellaire     Paella Restaurant    Spanish Restaurant   
20                Belle Harbor     Paella Restaurant    Spanish Restaurant   
21                   Bellerose             Wine Shop     Paella Restaurant   
22                     Belmont    Spanish Restaurant     Paella Restaurant   
23                 Bensonhurst     Paella Restaurant    Spanish Restaurant   
24                Bergen Beach     Paella Restaurant    Spanish Restaurant   
25                  Blissville     Paella Restaurant    Spanish Restaurant   
26                  Bloomfield     Paella Restaurant    Spanish Restaurant   
27                 Boerum Hill             Wine Shop     Paella Restaurant   
28                Borough Park     Paella Restaurant    Spanish Restaurant   
29                Breezy Point     Paella Restaurant    Spanish Restaurant   
30                   Briarwood     Paella Restaurant    Spanish Restaurant   
31              Brighton Beach     Paella Restaurant    Spanish Restaurant   
32               Broad Channel     Paella Restaurant    Spanish Restaurant   
33           Broadway Junction     Paella Restaurant    Spanish Restaurant   
34                   Bronxdale    Spanish Restaurant     Paella Restaurant   
35            Brooklyn Heights             Wine Shop              Wine Bar   
36                  Brookville     Paella Restaurant    Spanish Restaurant   
37                 Brownsville    Spanish Restaurant     Paella Restaurant   
38                  Bulls Head     Paella Restaurant    Spanish Restaurant   
39                    Bushwick     Paella Restaurant    Spanish Restaurant   
40                Butler Manor     Paella Restaurant    Spanish Restaurant   
41             Cambria Heights     Paella Restaurant    Spanish Restaurant   
42                    Canarsie     Paella Restaurant    Spanish Restaurant   
43               Carnegie Hill             Wine Shop              Wine Bar   
44             Carroll Gardens             Wine Shop              Wine Bar   
45                 Castle Hill     Paella Restaurant    Spanish Restaurant   
46           Castleton Corners     Paella Restaurant    Spanish Restaurant   
47              Central Harlem      Tapas Restaurant     Paella Restaurant   
48                  Charleston     Paella Restaurant    Spanish Restaurant   
49                     Chelsea    Spanish Restaurant      Tapas Restaurant   
50                   Chinatown    Spanis

In [16]:
### Cluster neighborhoods in order to analyze them

# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_spa_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([2, 0, 0, 0, 0, 4, 0, 0, 0, 2], dtype=int32)

In [17]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_spa_merged = neighborhoods

# merge ny_spa_merged with neighborhoods to add latitude/longitude for each neighborhood
ny_spa_merged = ny_spa_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [18]:
ny_spa_merged

Borough               Neighborhood   Latitude  Longitude  \
0            Bronx                  Wakefield  40.894705 -73.847201   
1            Bronx                 Co-op City  40.874294 -73.829939   
2            Bronx                Eastchester  40.887556 -73.827806   
3            Bronx                  Fieldston  40.895437 -73.905643   
4            Bronx                  Riverdale  40.890834 -73.912585   
5            Bronx                Kingsbridge  40.881687 -73.902818   
6        Manhattan                Marble Hill  40.876551 -73.910660   
7            Bronx                   Woodlawn  40.898273 -73.867315   
8            Bronx                    Norwood  40.877224 -73.879391   
9            Bronx             Williamsbridge  40.881039 -73.857446   
10           Bronx                 Baychester  40.866858 -73.835798   
11           Bronx             Pelham Parkway  40.857413 -73.854756   
12           Bronx                City Island  40.847247 -73.786488   
13           Bronx               Bedford Park  40.870185 -73.885512   
14           Bronx         University Heights  40.855727 -73.910416   
15           Bronx             Morris Heights  40.847898 -73.919672   
16           Bronx                    Fordham  40.860997 -73.896427   
17           Bronx               East Tremont  40.842696 -73.887356   
18           Bronx                 West Farms  40.839475 -73.877745   
19           Bronx               High  Bridge  40.836623 -73.926102   
20           Bronx                    Melrose  40.819754 -73.909422   
21           Bronx                 Mott Haven  40.806239 -73.916100   
22           Bronx                Port Morris  40.801664 -73.913221   
23           Bronx                   Longwood  40.815099 -73.895788   
24           Bronx                Hunts Point  40.809730 -73.883315   
25           Bronx                 Morrisania  40.823592 -73.901506   
26           Bronx                  Soundview  40.821012 -73.865746   
27           Bronx               Clason Point  40.806551 -73.854144   
28           Bronx                Throgs Neck  40.815109 -73.816350   
29           Bronx               Country Club  40.844246 -73.824099   
30           Bronx                Parkchester  40.837938 -73.856003   
31           Bronx         Westchester Square  40.840619 -73.842194   
32           Bronx                   Van Nest  40.843608 -73.866299   
33           Bronx                Morris Park  40.847549 -73.850402   
34           Bronx                    Belmont  40.857277 -73.888452   
35           Bronx             Spuyten Duyvil  40.881395 -73.917190   
36           Bronx            North Riverdale  40.908543 -73.904531   
37           Bronx                 Pelham Bay  40.850641 -73.832074   
38           Bronx              Schuylerville  40.826580 -73.826203   
39           Bronx             Edgewater Park  40.821986 -73.813885   
40           Bronx                Castle Hill  40.819014 -73.848027   
41           Bronx                  Olinville  40.871371 -73.863324   
42           Bronx             Pelham Gardens  40.862966 -73.841612   
43           Bronx                  Concourse  40.834284 -73.915589   
44           Bronx                  Unionport  40.829774 -73.850535   
45           Bronx                   Edenwald  40.884561 -73.848083   
46        Brooklyn                  Bay Ridge  40.625801 -74.030621   
47        Brooklyn                Bensonhurst  40.611009 -73.995180   
48        Brooklyn                Sunset Park  40.645103 -74.010316   
49        Brooklyn                 Greenpoint  40.730201 -73.954241   
50        Brooklyn                  Gravesend  40.595260 -73.973471   
51        Brooklyn             Brighton Beach  40.576825 -73.965094   
52        Brooklyn             Sheepshead Bay  40.586890 -73.943186   
53        Brooklyn          Manhattan Terrace  40.614433 -73.957438   
54        Brooklyn                   Flatbush  40.636326 -73.958401   
55        Brooklyn              Cro

In [21]:
# As Folium can display values only in INT type, I checked that "Clustes Labels are of type Int"

ny_spa_merged.dtypes

Borough                   object
Neighborhood              object
Latitude                 float64
Longitude                float64
Cluster Labels           float64
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
dtype: object

In [27]:
# After some digging I've found that Cluster Lablels is float64 due to some NaN values, which will be removed.

ny_spa_merged_noNaN = ny_spa_merged.dropna()

In [28]:
# This is the DF after the NaN cleaning

ny_spa_merged_noNaN

Borough               Neighborhood   Latitude  Longitude  \
0            Bronx                  Wakefield  40.894705 -73.847201   
1            Bronx                 Co-op City  40.874294 -73.829939   
2            Bronx                Eastchester  40.887556 -73.827806   
3            Bronx                  Fieldston  40.895437 -73.905643   
4            Bronx                  Riverdale  40.890834 -73.912585   
5            Bronx                Kingsbridge  40.881687 -73.902818   
6        Manhattan                Marble Hill  40.876551 -73.910660   
7            Bronx                   Woodlawn  40.898273 -73.867315   
8            Bronx                    Norwood  40.877224 -73.879391   
9            Bronx             Williamsbridge  40.881039 -73.857446   
10           Bronx                 Baychester  40.866858 -73.835798   
11           Bronx             Pelham Parkway  40.857413 -73.854756   
12           Bronx                City Island  40.847247 -73.786488   
13           Bronx               Bedford Park  40.870185 -73.885512   
14           Bronx         University Heights  40.855727 -73.910416   
15           Bronx             Morris Heights  40.847898 -73.919672   
16           Bronx                    Fordham  40.860997 -73.896427   
17           Bronx               East Tremont  40.842696 -73.887356   
18           Bronx                 West Farms  40.839475 -73.877745   
19           Bronx               High  Bridge  40.836623 -73.926102   
20           Bronx                    Melrose  40.819754 -73.909422   
21           Bronx                 Mott Haven  40.806239 -73.916100   
22           Bronx                Port Morris  40.801664 -73.913221   
23           Bronx                   Longwood  40.815099 -73.895788   
24           Bronx                Hunts Point  40.809730 -73.883315   
25           Bronx                 Morrisania  40.823592 -73.901506   
26           Bronx                  Soundview  40.821012 -73.865746   
27           Bronx               Clason Point  40.806551 -73.854144   
28           Bronx                Throgs Neck  40.815109 -73.816350   
29           Bronx               Country Club  40.844246 -73.824099   
30           Bronx                Parkchester  40.837938 -73.856003   
31           Bronx         Westchester Square  40.840619 -73.842194   
32           Bronx                   Van Nest  40.843608 -73.866299   
33           Bronx                Morris Park  40.847549 -73.850402   
34           Bronx                    Belmont  40.857277 -73.888452   
35           Bronx             Spuyten Duyvil  40.881395 -73.917190   
36           Bronx            North Riverdale  40.908543 -73.904531   
37           Bronx                 Pelham Bay  40.850641 -73.832074   
38           Bronx              Schuylerville  40.826580 -73.826203   
39           Bronx             Edgewater Park  40.821986 -73.813885   
40           Bronx                Castle Hill  40.819014 -73.848027   
41           Bronx                  Olinville  40.871371 -73.863324   
42           Bronx             Pelham Gardens  40.862966 -73.841612   
43           Bronx                  Concourse  40.834284 -73.915589   
44           Bronx                  Unionport  40.829774 -73.850535   
45           Bronx                   Edenwald  40.884561 -73.848083   
46        Brooklyn                  Bay Ridge  40.625801 -74.030621   
47        Brooklyn                Bensonhurst  40.611009 -73.995180   
48        Brooklyn                Sunset Park  40.645103 -74.010316   
49        Brooklyn                 Greenpoint  40.730201 -73.954241   
50        Brooklyn                  Gravesend  40.595260 -73.973471   
51        Brooklyn             Brighton Beach  40.576825 -73.965094   
52        Brooklyn             Sheepshead Bay  40.586890 -73.943186   
53        Brooklyn          Manhattan Terrace  40.614433 -73.957438   
54        Brooklyn                   Flatbush  40.636326 -73.958401   
55        Brooklyn              Cro

In [31]:
# Now that there are no NaN values, the Cluster Lables column type can be changed to type int
ny_spa_merged_noNaN[['Cluster Labels']] = ny_spa_merged_noNaN[['Cluster Labels']].astype(int)

# This is the DF proceseed ant ready to be displayed on Folium

ny_spa_merged_noNaN

/Users/sebasjunca/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Borough               Neighborhood   Latitude  Longitude  \
0            Bronx                  Wakefield  40.894705 -73.847201   
1            Bronx                 Co-op City  40.874294 -73.829939   
2            Bronx                Eastchester  40.887556 -73.827806   
3            Bronx                  Fieldston  40.895437 -73.905643   
4            Bronx                  Riverdale  40.890834 -73.912585   
5            Bronx                Kingsbridge  40.881687 -73.902818   
6        Manhattan                Marble Hill  40.876551 -73.910660   
7            Bronx                   Woodlawn  40.898273 -73.867315   
8            Bronx                    Norwood  40.877224 -73.879391   
9            Bronx             Williamsbridge  40.881039 -73.857446   
10           Bronx                 Baychester  40.866858 -73.835798   
11           Bronx             Pelham Parkway  40.857413 -73.854756   
12           Bronx                City Island  40.847247 -73.786488   
13           Bronx               Bedford Park  40.870185 -73.885512   
14           Bronx         University Heights  40.855727 -73.910416   
15           Bronx             Morris Heights  40.847898 -73.919672   
16           Bronx                    Fordham  40.860997 -73.896427   
17           Bronx               East Tremont  40.842696 -73.887356   
18           Bronx                 West Farms  40.839475 -73.877745   
19           Bronx               High  Bridge  40.836623 -73.926102   
20           Bronx                    Melrose  40.819754 -73.909422   
21           Bronx                 Mott Haven  40.806239 -73.916100   
22           Bronx                Port Morris  40.801664 -73.913221   
23           Bronx                   Longwood  40.815099 -73.895788   
24           Bronx                Hunts Point  40.809730 -73.883315   
25           Bronx                 Morrisania  40.823592 -73.901506   
26           Bronx                  Soundview  40.821012 -73.865746   
27           Bronx               Clason Point  40.806551 -73.854144   
28           Bronx                Throgs Neck  40.815109 -73.816350   
29           Bronx               Country Club  40.844246 -73.824099   
30           Bronx                Parkchester  40.837938 -73.856003   
31           Bronx         Westchester Square  40.840619 -73.842194   
32           Bronx                   Van Nest  40.843608 -73.866299   
33           Bronx                Morris Park  40.847549 -73.850402   
34           Bronx                    Belmont  40.857277 -73.888452   
35           Bronx             Spuyten Duyvil  40.881395 -73.917190   
36           Bronx            North Riverdale  40.908543 -73.904531   
37           Bronx                 Pelham Bay  40.850641 -73.832074   
38           Bronx              Schuylerville  40.826580 -73.826203   
39           Bronx             Edgewater Park  40.821986 -73.813885   
40           Bronx                Castle Hill  40.819014 -73.848027   
41           Bronx                  Olinville  40.871371 -73.863324   
42           Bronx             Pelham Gardens  40.862966 -73.841612   
43           Bronx                  Concourse  40.834284 -73.915589   
44           Bronx                  Unionport  40.829774 -73.850535   
45           Bronx                   Edenwald  40.884561 -73.848083   
46        Brooklyn                  Bay Ridge  40.625801 -74.030621   
47        Brooklyn                Bensonhurst  40.611009 -73.995180   
48        Brooklyn                Sunset Park  40.645103 -74.010316   
49        Brooklyn                 Greenpoint  40.730201 -73.954241   
50        Brooklyn                  Gravesend  40.595260 -73.973471   
51        Brooklyn             Brighton Beach  40.576825 -73.965094   
52        Brooklyn             Sheepshead Bay  40.586890 -73.943186   
53        Brooklyn          Manhattan Terrace  40.614433 -73.957438   
54        Brooklyn                   Flatbush  40.636326 -73.958401   
55        Brooklyn              Cro

In [44]:
# visualize the clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_spa_merged_noNaN['Latitude'], ny_spa_merged_noNaN['Longitude'], ny_spa_merged_noNaN['Neighborhood'], ny_spa_merged_noNaN['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster + 1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the Clusters

In [48]:
#Cluster 1

ny_cluster_1 = ny_spa_merged_noNaN.loc[ny_spa_merged_noNaN['Cluster Labels'] == 0, ny_spa_merged_noNaN.columns[[0, 1] + list(range(5, ny_spa_merged_noNaN.shape[1]))]]

ny_cluster_1.head()

Borough Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0   Bronx    Wakefield     Paella Restaurant    Spanish Restaurant   
1   Bronx   Co-op City     Paella Restaurant    Spanish Restaurant   
2   Bronx  Eastchester     Paella Restaurant    Spanish Restaurant   
3   Bronx    Fieldston     Paella Restaurant    Spanish Restaurant   
4   Bronx    Riverdale     Paella Restaurant    Spanish Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
0      Tapas Restaurant             Wine Shop              Wine Bar  
1      Tapas Restaurant             Wine Shop              Wine Bar  
2      Tapas Restaurant             Wine Shop              Wine Bar  
3      Tapas Restaurant             Wine Shop              Wine Bar  
4      Tapas Restaurant             Wine Shop              Wine Bar

In [49]:
#Cluster 2

ny_cluster_2 = ny_spa_merged_noNaN.loc[ny_spa_merged_noNaN['Cluster Labels'] == 1, ny_spa_merged_noNaN.columns[[0, 1] + list(range(5, ny_spa_merged_noNaN.shape[1]))]]

ny_cluster_2.head()

Borough    Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
15   Bronx  Morris Heights    Spanish Restaurant     Paella Restaurant   
17   Bronx    East Tremont     Paella Restaurant    Spanish Restaurant   
19   Bronx    High  Bridge    Spanish Restaurant     Paella Restaurant   
21   Bronx      Mott Haven    Spanish Restaurant     Paella Restaurant   
22   Bronx     Port Morris    Spanish Restaurant     Paella Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
15      Tapas Restaurant             Wine Shop              Wine Bar  
17      Tapas Restaurant             Wine Shop              Wine Bar  
19      Tapas Restaurant             Wine Shop              Wine Bar  
21      Tapas Restaurant             Wine Shop              Wine Bar  
22      Tapas Restaurant             Wine Shop              Wine Bar

In [50]:
#Cluster 3

ny_cluster_3 = ny_spa_merged_noNaN.loc[ny_spa_merged_noNaN['Cluster Labels'] == 2, ny_spa_merged_noNaN.columns[[0, 1] + list(range(5, ny_spa_merged_noNaN.shape[1]))]]

ny_cluster_3.head()

Borough  Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
5    Bronx   Kingsbridge    Spanish Restaurant     Paella Restaurant   
8    Bronx       Norwood    Spanish Restaurant     Paella Restaurant   
10   Bronx    Baychester    Spanish Restaurant     Paella Restaurant   
12   Bronx   City Island    Spanish Restaurant     Paella Restaurant   
13   Bronx  Bedford Park    Spanish Restaurant     Paella Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
5       Tapas Restaurant             Wine Shop              Wine Bar  
8       Tapas Restaurant             Wine Shop              Wine Bar  
10      Tapas Restaurant             Wine Shop              Wine Bar  
12      Tapas Restaurant             Wine Shop              Wine Bar  
13      Tapas Restaurant             Wine Shop              Wine Bar

In [51]:
#Cluster 4

ny_cluster_4 = ny_spa_merged_noNaN.loc[ny_spa_merged_noNaN['Cluster Labels'] == 3, ny_spa_merged_noNaN.columns[[0, 1] + list(range(5, ny_spa_merged_noNaN.shape[1]))]]

ny_cluster_4.head()

Borough        Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
61  Brooklyn        Williamsburg      Tapas Restaurant              Wine Bar   
63  Brooklyn  Bedford Stuyvesant             Wine Shop              Wine Bar   
69  Brooklyn         Fort Greene             Wine Shop              Wine Bar   
96  Brooklyn          North Side              Wine Bar             Wine Shop   
97  Brooklyn          South Side              Wine Bar      Tapas Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
61     Paella Restaurant    Spanish Restaurant             Wine Shop  
63     Paella Restaurant    Spanish Restaurant      Tapas Restaurant  
69      Tapas Restaurant     Paella Restaurant    Spanish Restaurant  
96     Paella Restaurant    Spanish Restaurant      Tapas Restaurant  
97     Paella Restaurant    Spanish Restaurant             Wine Shop

In [52]:
#Cluster 5

ny_cluster_5 = ny_spa_merged_noNaN.loc[ny_spa_merged_noNaN['Cluster Labels'] == 4, ny_spa_merged_noNaN.columns[[0, 1] + list(range(5, ny_spa_merged_noNaN.shape[1]))]]

ny_cluster_5.head()

Borough      Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
56  Brooklyn     East Flatbush             Wine Shop     Paella Restaurant   
58  Brooklyn   Windsor Terrace             Wine Shop     Paella Restaurant   
59  Brooklyn  Prospect Heights             Wine Shop              Wine Bar   
64  Brooklyn  Brooklyn Heights             Wine Shop              Wine Bar   
65  Brooklyn       Cobble Hill             Wine Shop    Spanish Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
56    Spanish Restaurant      Tapas Restaurant              Wine Bar  
58    Spanish Restaurant      Tapas Restaurant              Wine Bar  
59     Paella Restaurant    Spanish Restaurant      Tapas Restaurant  
64     Paella Restaurant    Spanish Restaurant      Tapas Restaurant  
65              Wine Bar     Paella Restaurant      Tapas Restaurant